## Extension of GTAA Timing Strategy (dynamic asset allocation) (1973-2012)

**Rules**
<br>
- *SAME AS GTAA TIMING MODEL*
- Now each of the 5 global assets are weighted dynamically depending on their momentum
- Calculate momentum over same period as the SMA at the end of each month and update weighting of model accordingly
- Highest momentum is assigned a rank of 5 --> weighting of 5/1+2+3+4+5 = 0.33%. Lowest momentum is assigned a rank of 1 --> weighting of 1/1+2+3+4+5 = 0.066%

*Types of Assets*
1. US stocks: S&P500 (SPY)
2. Foreign stocks: MSCI EAFE
3. Bonds: LUATTRUU
5. Commodities: BCOM
4. Real estate: NAREIT

In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

#CHANGE MOVING AVERAGE WINDOW BELOW (IN MONTHS)
MOVING_AVERAGE_WINDOW = 9
STARTING_BALANCE = 100
#CHANGE YEARS BELOW FOR MORE RECENT DATA (11) AND FOR MEB FABER'S DATA RANGE (40)
YEARS = 40
PERIODS = YEARS * 12

#load data into a pandas dataframe
#CHANGE NAME OF EXCEL FILES TO SPX_recent.csv FOR MORE RECENT DATA AND SPX_gtaa.csv FOR MEB FABER'S DATA RANGE
SPXprices = pd.read_csv('SPX_gtaa.csv', index_col='Date', parse_dates=['Date'])
MSCIEAFEprices = pd.read_csv('MSCIEAFE_gtaa.csv', index_col='Date', parse_dates=['Date'])
LUATTRUUprices = pd.read_csv('LUATTRUU_gtaa.csv', index_col='Date', parse_dates=['Date'])
BCOMTRprices = pd.read_csv('BCOMTR_gtaa.csv', index_col='Date', parse_dates=['Date'])
FNERTRprices = pd.read_csv('FNERTR_gtaa.csv', index_col='Date', parse_dates=['Date'])

#calculate moving average
SPXprices['SMA'] = SPXprices.Close.rolling(window = MOVING_AVERAGE_WINDOW).mean()
MSCIEAFEprices['SMA'] = MSCIEAFEprices.Close.rolling(window = MOVING_AVERAGE_WINDOW).mean()
LUATTRUUprices['SMA'] = LUATTRUUprices.Close.rolling(window = MOVING_AVERAGE_WINDOW).mean()
BCOMTRprices['SMA'] = BCOMTRprices.Close.rolling(window = MOVING_AVERAGE_WINDOW).mean()
FNERTRprices['SMA'] = FNERTRprices.Close.rolling(window = MOVING_AVERAGE_WINDOW).mean()

#calculate periodic return of each asset
SPXprices['Nominal_Returns'] = 1+SPXprices.NominalReturn
MSCIEAFEprices['Nominal_Returns'] = 1+MSCIEAFEprices.NominalReturn_MSCIEAFE
LUATTRUUprices['Nominal_Returns'] = 1+LUATTRUUprices.NominalReturn_LUATTRUU
BCOMTRprices['Nominal_Returns'] = 1+BCOMTRprices.NominalReturn_BCOMTR
FNERTRprices['Nominal_Returns'] = 1+FNERTRprices.NominalReturn_FNERTR

#calculate momentum
GTAAmomentum = pd.read_csv('GTAA_momentum.csv', index_col='Date', parse_dates=['Date'])
GTAAmomentum['SPX_Momentum'] = SPXprices['Nominal_Returns'].rolling(window = MOVING_AVERAGE_WINDOW).apply(lambda x: x.prod())
GTAAmomentum['MSCIEAFE_Momentum'] = MSCIEAFEprices['Nominal_Returns'].rolling(window = MOVING_AVERAGE_WINDOW).apply(lambda x: x.prod())
GTAAmomentum['LUATTRUU_Momentum'] = LUATTRUUprices['Nominal_Returns'].rolling(window = MOVING_AVERAGE_WINDOW).apply(lambda x: x.prod())
GTAAmomentum['BCOMTR_Momentum'] = BCOMTRprices['Nominal_Returns'].rolling(window = MOVING_AVERAGE_WINDOW).apply(lambda x: x.prod())
GTAAmomentum['FNERTR_Momentum'] = FNERTRprices['Nominal_Returns'].rolling(window = MOVING_AVERAGE_WINDOW).apply(lambda x: x.prod())

#calculate ranks based on momentum
ranks = GTAAmomentum.rank(axis=1, method='min', ascending=True)
for col in GTAAmomentum.columns:
    GTAAmomentum[f'{col}_Rank']=ranks[col]

#calculate weights based on rank
GTAAmomentum['SPX_Momentum_Weight'] = GTAAmomentum['SPX_Momentum_Rank']/15
GTAAmomentum['MSCIEAFE_Momentum_Weight'] = GTAAmomentum['MSCIEAFE_Momentum_Rank']/15
GTAAmomentum['LUATTRUU_Momentum_Weight'] = GTAAmomentum['LUATTRUU_Momentum_Rank']/15
GTAAmomentum['BCOMTR_Momentum_Weight'] = GTAAmomentum['BCOMTR_Momentum_Rank']/15
GTAAmomentum['FNERTR_Momentum_Weight'] = GTAAmomentum['FNERTR_Momentum_Rank']/15

GTAAmomentum.fillna(0.2, inplace=True)

#calculate new periodic return of each asset based on weights
SPXprices['Nominal_Returns_Weighted'] = SPXprices['Nominal_Returns']*GTAAmomentum['SPX_Momentum_Weight']
MSCIEAFEprices['Nominal_Returns_Weighted'] = MSCIEAFEprices['Nominal_Returns']*GTAAmomentum['MSCIEAFE_Momentum_Weight']
LUATTRUUprices['Nominal_Returns_Weighted'] = LUATTRUUprices['Nominal_Returns']*GTAAmomentum['LUATTRUU_Momentum_Weight']
BCOMTRprices['Nominal_Returns_Weighted'] = BCOMTRprices['Nominal_Returns']*GTAAmomentum['BCOMTR_Momentum_Weight']
FNERTRprices['Nominal_Returns_Weighted'] = FNERTRprices['Nominal_Returns']*GTAAmomentum['FNERTR_Momentum_Weight']

GTAAmomentum.tail()

C:\Users\dinga\AppData\Local\Temp\ipykernel_22808\1818023573.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  SPXprices = pd.read_csv('SPX_gtaa.csv', index_col='Date', parse_dates=['Date'])
C:\Users\dinga\AppData\Local\Temp\ipykernel_22808\1818023573.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  MSCIEAFEprices = pd.read_csv('MSCIEAFE_gtaa.csv', index_col='Date', parse_dates=['Date'])
C:\Users\dinga\AppData\Local\Temp\ipykernel_22808\1818023573.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  LUATTRUUprices = pd.read_csv('LUATTRUU_gtaa.csv', index_col='Date', 

,SPX_Momentum,MSCIEAFE_Momentum,LUATTRUU_Momentum,BCOMTR_Momentum,FNERTR_Momentum,SPX_Momentum_Rank,MSCIEAFE_Momentum_Rank,LUATTRUU_Momentum_Rank,BCOMTR_Momentum_Rank,FNERTR_Momentum_Rank,SPX_Momentum_Weight,MSCIEAFE_Momentum_Weight,LUATTRUU_Momentum_Weight,BCOMTR_Momentum_Weight,FNERTR_Momentum_Weight
Date,,,,,,,,,,,,,,,
2012-08-01,1.160042,1.062220,1.033822,0.999645,1.231912,4.0,3.0,2.0,1.0,5.0,0.266667,0.200000,0.133333,0.066667,0.333333
2012-09-01,1.167751,1.104361,1.020716,1.056353,1.161030,5.0,3.0,1.0,2.0,4.0,0.333333,0.200000,0.066667,0.133333,0.266667
2012-10-01,1.116795,1.057079,1.014719,0.990994,1.088875,5.0,3.0,2.0,1.0,4.0,0.333333,0.200000,0.133333,0.066667,0.266667
2012-11-01,1.050504,1.023599,1.027289,0.965423,1.095908,4.0,2.0,3.0,1.0,5.0,0.266667,0.133333,0.200000,0.066667,0.333333
2012-12-01,1.053277,1.060699,1.033204,0.980832,1.083468,3.0,4.0,2.0,1.0,5.0,0.200000,0.266667,0.133333,0.066667,0.333333


In [4]:
#calculate whole portfolio returns and balance for dynamic weighting
#CHANGE NAME OF EXCEL FILE TO GTAA_balance_recent.csv FOR MORE RECENT DATA AND GTAA_balance.csv FOR MEB FABER'S DATA RANGE
GTAA_balance = pd.read_csv('GTAA_balance.csv', index_col='Date', parse_dates=['Date'])

#create signals
SPXprices['Buy'] = SPXprices.Close > SPXprices.SMA
MSCIEAFEprices['Buy'] = MSCIEAFEprices.Close > MSCIEAFEprices.SMA
LUATTRUUprices['Buy'] = LUATTRUUprices.Close > LUATTRUUprices.SMA
BCOMTRprices['Buy'] = BCOMTRprices.Close > BCOMTRprices.SMA
FNERTRprices['Buy'] = FNERTRprices.Close > FNERTRprices.SMA

#read tbill data
#CHANGE NAME OF EXCEL FILE TO TBills_recent.csv FOR MORE RECENT DATA AND TBills_gtaa.csv FOR MEB FABER'S DATA RANGE
tbill_gtaa = pd.read_csv('TBills_gtaa.csv', index_col='Index', parse_dates=['Index'])
GTAAmomentum['DTB3CloseReturns_SPX'] = (1+(tbill_gtaa.DTB3Close))*GTAAmomentum['SPX_Momentum_Weight']
GTAAmomentum['DTB3CloseReturns_MSCIEAFE'] = (1+(tbill_gtaa.DTB3Close))*GTAAmomentum['MSCIEAFE_Momentum_Weight']
GTAAmomentum['DTB3CloseReturns_LUATTRUU'] = (1+(tbill_gtaa.DTB3Close))*GTAAmomentum['LUATTRUU_Momentum_Weight']
GTAAmomentum['DTB3CloseReturns_BCOMTR'] = (1+(tbill_gtaa.DTB3Close))*GTAAmomentum['BCOMTR_Momentum_Weight']
GTAAmomentum['DTB3CloseReturns_FNERTR'] = (1+(tbill_gtaa.DTB3Close))*GTAAmomentum['FNERTR_Momentum_Weight']

#calculate periodic return
#ADD DELAY BELOW 
#data['Strategy_Return'] = np.where(data.Buy.shift(2) == True, data['Nominal_Returns'], tbill.DTB3CloseReturns)
SPXprices['Strategy_Return'] = np.where(SPXprices.Buy.shift(1) == True, SPXprices['Nominal_Returns_Weighted'], GTAAmomentum.DTB3CloseReturns_SPX)
MSCIEAFEprices['Strategy_Return'] = np.where(MSCIEAFEprices.Buy.shift(1)== True, MSCIEAFEprices['Nominal_Returns_Weighted'], GTAAmomentum.DTB3CloseReturns_MSCIEAFE)
LUATTRUUprices['Strategy_Return'] = np.where(LUATTRUUprices.Buy.shift(1)== True, LUATTRUUprices['Nominal_Returns_Weighted'], GTAAmomentum.DTB3CloseReturns_LUATTRUU)
BCOMTRprices['Strategy_Return'] = np.where(BCOMTRprices.Buy.shift(1)== True, BCOMTRprices['Nominal_Returns_Weighted'], GTAAmomentum.DTB3CloseReturns_BCOMTR)
FNERTRprices['Strategy_Return'] = np.where(FNERTRprices.Buy.shift(1) == True, FNERTRprices['Nominal_Returns_Weighted'], GTAAmomentum.DTB3CloseReturns_FNERTR)

#calculate whole portfolio returns and balance
GTAA_balance['GTAA_Strategy_Return'] = SPXprices['Strategy_Return'] + MSCIEAFEprices['Strategy_Return'] + LUATTRUUprices['Strategy_Return'] + BCOMTRprices['Strategy_Return'] + FNERTRprices['Strategy_Return'] 
GTAA_balance['GTAA_Strategy_Balance'] = STARTING_BALANCE*GTAA_balance.GTAA_Strategy_Return.cumprod()

#calculate compounded annualized return
total_strategy_cagr = round((((GTAA_balance['GTAA_Strategy_Return'].product()) ** (12/PERIODS))-1) * 100, 2)
print(f'PORTFOLIO TOTAL CAGR (DYNAMIC MOVING AVERAGE TIMING): {total_strategy_cagr}%')

#calculate drawdowns of whole portfolio
GTAA_balance['GTAA_Strategy_balance_Peak'] = GTAA_balance.GTAA_Strategy_Balance.cummax()
GTAA_balance['GTAA_Strategy_balance_DD'] = GTAA_balance.GTAA_Strategy_Balance - GTAA_balance.GTAA_Strategy_balance_Peak
GTAA_Strategy_balance_dd = round((((GTAA_balance.GTAA_Strategy_balance_DD / GTAA_balance.GTAA_Strategy_balance_Peak).min()) * 100), 2)
print(f'GTAA MAX DD (DYNAMIC MOVING AVERAGE TIMING): {GTAA_Strategy_balance_dd}%')

PORTFOLIO TOTAL CAGR (DYNAMIC MOVING AVERAGE TIMING): 14.84%
GTAA MAX DD (DYNAMIC MOVING AVERAGE TIMING): -11.54%


C:\Users\dinga\AppData\Local\Temp\ipykernel_22808\2813066699.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  GTAA_balance = pd.read_csv('GTAA_balance.csv', index_col='Date', parse_dates=['Date'])
C:\Users\dinga\AppData\Local\Temp\ipykernel_22808\2813066699.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tbill_gtaa = pd.read_csv('TBills_gtaa.csv', index_col='Index', parse_dates=['Index'])


### Extension of GTAA timing strategy Conclusion (1973-2012)

**EQUAL WEIGHTING GTAA MODEL (9 MONTH SMA):**
- Buy & Hold CAGR: 9.47%
- Buy & Hold MAX DD: -48.77%

- Strategy CAGR: 10.14%
- Strategy MAX DD: -18.66%

**DYNAMIC WEIGHTING GTAA MODEL (9 MONTH SMA):**
- Buy & Hold CAGR: 9.47%
- Buy & Hold MAX DD: -48.77%

- Strategy CAGR: 14.84%
- Strategy MAX DD: -11.54%